In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyedfread import edf
import pickle
from glob import glob
from os.path import join
%matplotlib inline

In [3]:
original = '/Volumes/psp_data/PSP/data/raw/GaitRite/GaitGrid_all_PSP_IPS_Controll.csv' # File send by Marc

file = '/Volumes/psp_data/PSP/data/raw/GaitRite/GAITRite_data_2.tsv' # original only slighlty adjusted columns

In [4]:
gait = pd.read_table('/Volumes/psp_data/PSP/data/raw/GaitRite/GAITRite_data_2.tsv', header=[0, 1], index_col=[0, 1, 2], decimal=",")


In [8]:
gait.head()

normal                               \
PatientenID Distance (cm) normal Ambulation Time (sec) normal   
1 1 1.0               512.903333                     9.960000   
2 1 1.0               535.936667                     8.016667   
3 1 2.5               526.136667                     5.286667   
6 1 2.0               543.610000                    17.610000   
7 1 1.0               487.040000                     4.680000   

                                                                       \
PatientenID Velocity (cm/sec) normal Mean Normalized Velocity  normal   
1 1 1.0                    52.600000                         0.466667   
2 1 1.0                    68.000000                         0.643333   
3 1 2.5                    99.533333                         0.970000   
6 1 2.0                    30.900000                         0.303333   
7 1 1.0                   104.133333                         0.950000   

                                                                \
PatientenID Number of Steps  normal Cadence (Steps/Min) normal   
1 1 1.0                   12.000000                  73.000000   
2 1 1.0                   12.000000                  90.766667   
3 1 2.5                    9.000000                 102.166667   
6 1 2.0                   21.666667                  73.866667   
7 1 1.0                    8.000000                 102.566667   

                                                 \
PatientenID Step Time Differential (sec) normal   
1 1 1.0                                0.050000   
2 1 1.0                                0.153333   
3 1 2.5                                0.033333   
6 1 2.0                                0.046667   
7 1 1.0                                0.023333   

                                                  \
PatientenID Step Length Differential (cm) normal   
1 1 1.0                                 4.976667   
2 1 1.0                                 3.670000   
3 1 2.5                                 3.253333   
6 1 2.0                                 6.590000   
7 1 1.0                                 5.653333   

                                                                   ...  \
PatientenID Cycle Time Differential (sec) normal FAP score normal  ...   
1 1 1.0                                 0.026667        50.333333  ...   
2 1 1.0                                 0.020000        69.000000  ...   
3 1 2.5                                 0.010000        90.666667  ...   
6 1 2.0                                 0.013333        48.333333  ...   
7 1 1.0                                 0.010000        87.000000  ...   

                                   turn                                 \
PatientenID Double Support (%GC) R turn Double Support (%GC) mean turn   
1 1 1.0                       39.666667                      40.766667   
2 1 1.0                       43.700000                      35.475000   
3 1 2.5                       30.066667                      30.016667   
6 1 2.0                       60.933333                      55.933333   
7 1 1.0                       27.333333                      27.750000   

                                                                         \
PatientenID Swing (%GC) L turn Swing (%GC) R turn Swing (%GC) mean turn   
1 1 1.0              30.666667          27.033333             28.850000   
2 1 1.0              43.750000          26.950000             35.350000   
3 1 2.5              35.466667          35.400000             35.433333   
6 1 2.0              30.633333          30.066667             30.350000   
7 1 1.0              37.133333          35.400000             36.266667   

                                                                               \
PatientenID Stance (%GC) L turn Stance (%GC) R turn Toe In / Out (deg) L turn   
1 1 1.0               69.333333           72.966667                 19.133333   
2 1 1.0               30.700000           73.050000              

#### Comment Monika: velocity and cadence in PSP/RS in dual task better than in fast -> check

In [6]:
pat_data = pd.read_csv('/Users/kenohagena/Documents/Forschung/PSP/data/meta/Patientencharakteristika_Auswertung_Pupillometrie_Marc.csv', 
                       delimiter=';')
pat_data = pat_data.rename(columns={'ID ProPSP': 'subject',
                        'Alter Erstuntersuchung': 'age',
                        'Erkrankungsdauer (Jahre)': 'disease_duration',
                        'PSP Rating Scale': 'psp_rating_scale'}).drop('Patientennummer Pupillometrie', axis=1)

In [24]:
pat_data

,subject,Geschlecht,age,disease_duration,Subtypen,psp_rating_scale,MoCA
0,1,m,63,"1,5",PSP-RS,43,25
1,2,m,77,3,PSP-RS,39,21
2,3,m,82,2,PSP-P/F,27,24
3,6,w,78,6,PSP-P,39,23
4,7,m,81,"1,5",PSP-RS,32,20
5,9,m,63,7,PSP-P/F,24,10
6,10,m,60,2,PSP-RS,35,23
7,12,w,72,1,PSP-RS,23,27
8,13,m,72,4,PSP-P,19,27
9,14,w,71,"5,5",PSP-RS,62,14


In [48]:
rs = pat_data.loc[pat_data.Subtypen.str.startswith('PSP-RS')].subject.values
non_rs = pat_data.loc[~pat_data.Subtypen.str.startswith('PSP-RS')].subject.values

In [49]:
rs.shape

(14,)

In [50]:
non_rs.shape

(6,)

In [51]:
rs

array([ 1,  2,  7, 10, 12, 14, 16, 18, 19, 20, 21, 23, 25, 26])

In [52]:
non_rs

array([ 3,  6,  9, 13, 22, 24])

In [42]:
psp_with_gait = gait.reset_index()['level_0'][:15].values
psp_with_gait

array(['1', '2', '3', '6', '7', '9', '10', '12', '13', '16', '19', '20',
       '21', '22', '24'], dtype=object)

In [55]:
rs_with_gait = np.intersect1d(rs.astype(int), psp_with_gait.astype(int))
rs_with_gait

array([ 1,  2,  7, 10, 12, 16, 19, 20, 21])

In [56]:
non_rs_with_gait = np.intersect1d(non_rs.astype(int), psp_with_gait.astype(int))
non_rs_with_gait

array([ 3,  6,  9, 13, 22, 24])

In [71]:
gait_rs = gait.reset_index().loc[gait.reset_index().level_0.isin(rs_with_gait.astype(str))]

In [85]:
gait_rs.loc[:, 'normal'].mean()

PatientenID
Distance (cm) normal                    518.380370
Ambulation Time (sec) normal              7.005556
Velocity (cm/sec) normal                 78.362963
Mean Normalized Velocity  normal          0.598519
Number of Steps  normal                  10.407407
Cadence (Steps/Min) normal               91.177778
Step Time Differential (sec) normal       0.048889
Step Length Differential (cm) normal      4.331481
Cycle Time Differential (sec) normal      0.017778
FAP score normal                         70.851852
Step Time (sec) L normal                  0.661111
Step Time (sec) R normal                  0.671481
Step Time (sec) mean normal               0.666296
Cycle Time (sec) L normal                 1.326296
Cycle Time (sec) R normal                 1.321852
Cycle Time (sec) mean normal              1.324074
Step Length (cm) L normal                50.554815
Step Length (cm) R normal                51.724815
Step Length (cm) mean normal             51.139815
Step Length (cm) be

In [84]:
gait_rs.loc[:, 'turn'].mean()

PatientenID
Distance (cm) turn                    328.165556
Ambulation Time (sec) turn              4.622593
Velocity (cm/sec) turn                 76.425926
Mean Normalized Velocity  turn          0.680556
Number of Steps turn                    6.833333
Cadence (Steps/Min) turn               91.251852
Step Time Differential (sec) turn       0.060926
Step Length Differential (cm) turn      5.290926
Cycle Time Differential (sec) turn      0.044444
FAP score turn                         71.314815
Step Time (sec) L turn                  0.658333
Step Time (sec) R turn                  0.652593
Step Time (sec) mean turn               0.657222
Cycle Time (sec) L turn                 1.239074
Cycle Time (sec) R turn                 1.343333
Cycle Time (sec) mean turn              1.291204
Step Length (cm) L turn                44.968148
Step Length (cm) R turn                50.044815
Step Length (cm) mean turn             47.506481
Step Length (cm) best turn             51.359630
Step Len